## Instacart Market Analysis

    
<br>

Use this anonymized data on customer orders over time to predict which previously purchased products will be in a user’s next order. 

***


In [2]:
# load libraries
import numpy as np
import pandas as pd

In [ ]:
# import data
orders = pd.read_csv("./orders.csv")
order_prior = pd.read_csv("./order_product_prior.csv")
order_train = pd.read_csv("./order_product_train.csv")
aisles = pd.read_csv("./aisles.csv")
departments = pd.read_csv("./departments.csv")